In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

import plotly.graph_objects as go

from research.option_pricing import BlackScholesGreeks, BlackScholesModel


In [3]:
# Set the ticker symbol for the stock (e.g., Apple)
ticker = yf.Ticker("NVDA")  # Replace "AAPL" with your desired ticker symbol

# Get the expiration dates for available options
expiration_dates = ticker.options

In [6]:
import quantstream as qs

# Get the current stock price
fmp = qs.FmpConnector()

In [10]:
nvda = fmp.get_daily(
    symbol="NVDA",
    from_date="2023-01-01",
    to_date="2024-12-31",
)

In [11]:
nvda

<xarray.FinDataset> Size: 74kB
Dimensions:           (time: 460)
Coordinates:
  * time              (time) datetime64[ns] 4kB 2023-01-03 ... 2024-10-30
Data variables:
    open              (time) float64 4kB 14.85 14.57 14.49 ... 143.0 140.3 139.5
    high              (time) float64 4kB 15.0 14.85 14.56 ... 143.1 142.3 140.3
    low               (time) float64 4kB 14.1 14.24 14.15 ... 140.1 138.9 136.8
    close             (time) float64 4kB 14.32 14.75 14.27 ... 140.5 141.2 139.2
    adjClose          (time) float64 4kB 14.31 14.74 14.26 ... 140.5 141.2 139.2
    volume            (time) int64 4kB 401277000 431324000 ... 161917686
    unadjustedVolume  (time) int64 4kB 401277000 431324000 ... 161917686
    change            (time) float64 4kB -0.536 0.182 -0.226 ... 0.965 -0.36
    changePercent     (time) float64 4kB -3.57 1.24 -1.52 ... 0.6843 -0.258
    vwap              (time) float64 4kB 14.57 14.6 14.37 ... 141.7 140.7 139.0
    label             (time) <U16 29kB 'January 03, 23' ... 'October 30, 24'
    changeOverTime    (time) float64 4kB -0.0357 0.0124 ... 0.006843 -0.00258

In [26]:
# convert to dataframe
nvdf = nvda.to_dataframe()

In [27]:
nvdf

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
time,,,,,,,,,,,,
2023-01-03,14.85,15.00,14.10,14.32,14.31,401277000,401277000,-0.536,-3.57000,14.5675,"January 03, 23",-0.035700
2023-01-04,14.57,14.85,14.24,14.75,14.74,431324000,431324000,0.182,1.24000,14.6025,"January 04, 23",0.012400
2023-01-05,14.49,14.56,14.15,14.27,14.26,389168110,389168110,-0.226,-1.52000,14.3675,"January 05, 23",-0.015200
2023-01-06,14.47,15.01,14.03,14.86,14.85,405044000,405044000,0.385,2.70000,14.5925,"January 06, 23",0.027000
2023-01-09,15.28,16.06,15.14,15.63,15.62,504231000,504231000,0.344,2.29000,15.5275,"January 09, 23",0.022900
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-24,140.82,141.35,138.46,140.41,140.41,172354900,172354900,-0.410,-0.29115,140.2600,"October 24, 24",-0.002912
2024-10-25,140.93,144.13,140.80,141.54,141.54,205122109,205122109,0.610,0.43284,141.8500,"October 25, 24",0.004328
2024-10-28,143.00,143.14,140.05,140.52,140.52,173586745,173586745,-2.480,-1.73000,141.6775,"October 28, 24",-0.017300


In [28]:
# find golden cross
nvdf['SMA_50'] = nvdf['close'].rolling(window=50).mean()
nvdf['SMA_200'] = nvdf['close'].rolling(window=200).mean()

# Create a new column to identify golden cross
nvdf['Golden_Cross'] = np.where(nvdf['SMA_50'] > nvdf['SMA_200'], 1, 0)

# Create a new column to identify death cross
nvdf['Death_Cross'] = np.where(nvdf['SMA_50'] < nvdf['SMA_200'], 1, 0)

# Create a new column to identify crossovers
nvdf['Crossover'] = nvdf['Golden_Cross'].diff()

# Filter for golden cross and death cross dates
golden_cross_dates = nvdf[nvdf['Crossover'] == 1].index
death_cross_dates = nvdf[nvdf['Crossover'] == -1].index

# Print the dates of golden and death crosses
print("Golden Cross Dates:")
print(golden_cross_dates)

print("Death Cross Dates:")
print(death_cross_dates)

# Plotting the stock price and moving averages
fig = go.Figure()

# Add stock price trace
fig.add_trace(go.Scatter(x=nvdf.index, y=nvdf['close'], mode='lines', name='NVDA Price'))

# Add SMA traces
fig.add_trace(go.Scatter(x=nvdf.index, y=nvdf['SMA_50'], mode='lines', name='SMA 50'))
fig.add_trace(go.Scatter(x=nvdf.index, y=nvdf['SMA_200'], mode='lines', name='SMA 200'))

# Add vertical lines for golden and death crosses
for date in golden_cross_dates:
    fig.add_vline(x=date, line_color='green', line_width=2, line_dash='dash', name='Golden Cross')

for date in death_cross_dates:
    fig.add_vline(x=date, line_color='red', line_width=2, line_dash='dash', name='Death Cross')

# Update layout
fig.update_layout(title='NVDA Price with Golden and Death Crosses',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  legend_title='Legend')

# Show the plot
fig.show()

Golden Cross Dates:
DatetimeIndex(['2023-10-18'], dtype='datetime64[ns]', name='time', freq=None)
Death Cross Dates:
DatetimeIndex([], dtype='datetime64[ns]', name='time', freq=None)


In [29]:
# find distribution of returns
nvdf['Returns'] = nvdf['close'].pct_change()
nvdf['Volatility'] = nvdf['Returns'].rolling(window=21).std() * np.sqrt(252)  # Annualized volatility

# Calculate the mean and standard deviation of returns
mean_return = nvdf['Returns'].mean()
std_return = nvdf['Returns'].std()

# Print the mean and standard deviation of returns
print("Mean Return:", mean_return)
print("Standard Deviation of Returns:", std_return)

# Calculate the Sharpe Ratio
risk_free_rate = 0.01  # Example risk-free rate
sharpe_ratio = (mean_return - risk_free_rate) / std_return
print("Sharpe Ratio:", sharpe_ratio)

# Calculate the maximum drawdown
nvdf['Cumulative_Returns'] = (1 + nvdf['Returns']).cumprod()
nvdf['Drawdown'] = nvdf['Cumulative_Returns'] / nvdf['Cumulative_Returns'].cummax() - 1
max_drawdown = nvdf['Drawdown'].min()
print("Maximum Drawdown:", max_drawdown)

# Calculate the Value at Risk (VaR)
confidence_level = 0.95
var = -nvdf['Returns'].quantile(1 - confidence_level)
print("Value at Risk (VaR):", var)

Mean Return: 0.005477226436174783
Standard Deviation of Returns: 0.032427473928113154
Sharpe Ratio: -0.13947350860111793
Maximum Drawdown: -0.2704676205930082
Value at Risk (VaR): 0.03895994538006367


In [32]:
# plot histogram of returns
fig2 = go.Figure()
fig2.add_trace(go.Histogram(x=nvdf['Returns'].dropna(), nbinsx=200, name='Returns'))
fig2.update_layout(title='Histogram of Returns', xaxis_title='Returns', yaxis_title='Frequency')
fig2.show()

In [34]:
# kde plot of returns
fig3 = go.Figure()
fig3.add_trace(go.Histogram(x=nvdf['Returns'].dropna(), nbinsx=200, name='Returns', histnorm='probability density'))
fig3.update_layout(title='KDE of Returns', xaxis_title='Returns', yaxis_title='Density')
# plot normal curve
x = np.linspace(nvdf['Returns'].min(), nvdf['Returns'].max(), 100)
y = (1 / (std_return * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean_return) / std_return) ** 2)
fig3.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Normal Distribution', line=dict(color='red')))

# Show the plot
fig3.show()